In [23]:
import numpy as np
def poly(n,prime,r,p):
    ind = int(n/p)
    rem = n%p
    ans = rem
    i = 1
    while(ind != 0 and i <= r):
        ans+=(ind%p)*(prime**i)
        i+=1
        ind = int(ind/p)    
    return ans%p

def devore(p,r):
    if r>=p:
        print('r should be less than p')
        return None
    else:
        matrix = np.zeros((p*p, p**(r+1)),dtype='int')    
        for i in range(p**(r+1)):
            for j in range(p):
                val = poly(i,j,r,p)
                row_num = val + j*p
                matrix[row_num][i] = 1
        return matrix


In [24]:
p = 19
r = 4
mat = devore(p,r)
mat

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [25]:
mat.shape

(361, 2476099)

In [29]:
from math import *
mat = mat/sqrt(p)
N = mat.shape[1]
s = 3
x = np.zeros(N)
S1 = np.random.permutation(N)
S1 = S1[:s]
x[S1] = np.random.rand(s)
x = x/np.linalg.norm(x)
#compute y
y = mat@x
print (np.linalg.norm(y), np.linalg.norm(x))

1.000340566279137 1.0


In [27]:
#OMP
S = []
vec = np.zeros(N)
n_iter = s + 1
for i in range(n_iter):
    y_n = mat@vec
    j = np.argmax([abs(ele) for ele in mat.transpose()@(y - y_n)])
    if j not in S:
        S.append(j)
    x_n = np.linalg.pinv(mat[:,S])@y   
    vec = np.zeros(N)
    vec[S] = x_n

np.linalg.norm(x-vec)

1.5716250041522723e-16

In [30]:
#IHT
n_iter = 20
vec = np.zeros(N)
for i in range(n_iter):
    y_n = mat@vec
    x_n = vec + mat.transpose()@(y-y_n)
    vec = np.zeros(N)
    temp = sorted(x_n, key = abs)[-s:]
    vec[[np.argwhere(x_n == temp[i])[0][0] for i in range(s)]] = temp

  
np.linalg.norm(x-vec,2)

9.871997943141294e-16

In [31]:
#Convergence of OMP
n_trials = 50
omp_iter = []
for j in range(n_trials):
    x = np.zeros(N)
    S1 = np.random.permutation(N)
    S1 = S1[:s]
    x[S1] = np.random.rand(s)
    x = x/np.linalg.norm(x)
    y = mat@x
    S = []
    vec = np.zeros(N)
    n_iter = 2*s
    for i in range(n_iter):
        y_n = mat@vec
        j = np.argmax([abs(ele) for ele in mat.transpose()@(y - y_n)])
        if j not in S:
            S.append(j)
        x_n = np.linalg.pinv(mat[:,S])@y 
        vec = np.zeros(N)
        vec[S] = x_n
        if np.linalg.norm(x-vec) < 1e-12:
            omp_iter.append(i)
            break
    



In [32]:
#Convergence of IHT
n_trials = 50
iht_iter = []
for j in range(n_trials):    
    x = np.zeros(N)
    S1 = np.random.permutation(N)
    S1 = S1[:s]
    x[S1] = np.random.rand(s)
    x = x/np.linalg.norm(x)
    y = mat@x
    S = []
    vec = np.zeros(N)
    n_iter = 25
    vec = np.zeros(N)
    for i in range(n_iter):
        y_n = mat@vec
        x_n = vec + mat.transpose()@(y-y_n)
        vec = np.zeros(N)
        temp = sorted(x_n, key = abs)[-s:]
        vec[[np.argwhere(x_n == temp[i])[0][0] for i in range(s)]] = temp
        if np.linalg.norm(x-vec) < 1e-12:
            iht_iter.append(i)
            break

In [33]:
max(iht_iter)

20

In [34]:
max(omp_iter)

2